In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import streamlit as st

from utils import *

In [4]:
courses = []
certificates = []

course_numbers = [
    1043, 1045, 1046, 1047, 1048, 1049, 1050, 1054, 1055, 1056, 
    1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 
    1067, 1068, 1069, 1221
]

for n in tqdm(course_numbers):
    new_course = Course(f'https://certification.adobe.com/courses/{n}')
    courses.append(new_course)

print(len(courses))

certificate_htmls_location = 'certificate_htmls'

for html in tqdm(os.listdir(certificate_htmls_location)):
    certificate = Certificate(f'{certificate_htmls_location}/{html}')
    certificates.append(certificate)

100%|██████████| 24/24 [00:14<00:00,  1.64it/s]


24


100%|██████████| 54/54 [00:15<00:00,  3.45it/s]


In [20]:
def create_graph(course_name):
    H = nx.DiGraph()

    sources = [node for node in courses + certificates if node.category == course_name]

    for src in sources:
        H.add_node(src)

    for i, src1 in enumerate(sources):
        for j, src2 in enumerate(sources):
            if i == j:
                continue
            if src1.is_prereq_to(src2):
                H.add_edge(src1, src2, color='black')
    return H

def graph_to_2d_array(G):
    root_nodes = [n for n in G.nodes if G.in_degree(n) == 0]

    row_map = {}
    queue = [(node, 0) for node in root_nodes]

    while queue:
        node, row = queue.pop(0)

        if node in row_map:
            row = max(row, row_map[node])
        row_map[node] = row

        for child in G.successors(node):
            if child in row_map:
                row_map[child] = max(row_map[child], row + 1)
            else:
                row_map[child] = row + 1
            queue.append((child, row_map[child]))

    max_row = max(row_map.values())
    graph_2d = [[] for _ in range(max_row + 1)]

    for node, row in row_map.items():
        if type(node) == Course:
            node_type = 'course'
            desc = f'Course objectives: {node.objectives}'
        else:
            node_type = 'certificate'
            desc = f'Prerequisites: {node.prereq}'
        graph_2d[row].append({'type': node_type, 'display': node.display, 'description': desc})

    edges_2d = []
    for u, v in G.edges():
        if row_map[u] < row_map[v]:  # Ensure edges go downward
            edges_2d.append({'from': u.display, 'to': v.display})

    return graph_2d, edges_2d

In [21]:
course_category = 'Adobe Commerce'

G = create_graph(course_category)
nodes, edges = graph_to_2d_array(G)

In [39]:
def flatten(l):
    return [
        src
        for row in l
        for src in row 
    ]

def display_nodes(nodes):
    res = 'The relevant sources (nodes) are as follows: \n'
    for src in flatten(nodes):
        res += f'{src["type"]} - {src["display"]} - {src["description"]}\n'
    return res

def display_edges(edges):
    res = 'These nodes are connected via a directed edge as follows: \n'
    for edge in edges:
        res += f'From: {edge["from"]}, To: {edge["to"]}\n'
    return res

def get_full_string(nodes, edges):
    return display_nodes(nodes) + display_edges(edges)

In [40]:
get_full_string(nodes, edges)

"The relevant sources (nodes) are as follows: \ncourse - Adobe Commerce Foundations - Course objectives: This course provides foundational training covering essential product basics, program functionality, critical use cases, and real-world testimonials. Whether you're new to e-commerce or an experienced user, this course supplies you with the knowledge and insights needed to understand the dynamic world of Adobe Commerce. Just so you know, each module is separate. The system will track your progression through the course. Still, you must select each separate module, complete it, and then move to the next module. Once all the modules in this course are completed, you will earn your course completion certificate.\ncertificate - Adobe Commerce Business Practitioner Professional - Prerequisites: You should have a minimum of 0-12 months hands-on experience with Magento Open Source version 2.4.3 using the LUMA or BLANK theme and basic understanding of Adobe Commerce version 2.4.3., Addition